In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_csv('ToyotaCorolla - MLR.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ToyotaCorolla - MLR.csv'

# **EDA**

In [ ]:
data.shape

# Categorical Columns


## 1.Fuel_Type,2.HP,3.Automatic,4.cc,5.Doors,6.Cylinders,7.Gears

In [ ]:
## Fuel_Type

print(data['Fuel_Type'].unique())
print(data['Fuel_Type'].value_counts())                                          ## Imbalanced data [Petrol cars are too many][CNG car data are too less]

In [ ]:
## HP

print(data['HP'].unique())
print(data['HP'].value_counts().sort_values())                                  # skewed data  ## maybe we can remove the rows with HP 71,73,98

In [ ]:
## Automatic

print(data['Automatic'].unique())
print(data['Automatic'].value_counts().sort_values())                            # skewed data

In [ ]:
## cc

print(data['cc'].unique())
print(data['cc'].value_counts().sort_values())                                    # 16000 cc row outlier can be removed         ## maybe we can remove rows with cc =1975 ,1995,

In [ ]:
## Doors

print(data['Doors'].unique())
print(data['Doors'].value_counts().sort_values())                                  ## maybe we can remove cars with 2 doors

In [ ]:
## Cylinders

print(data['Cylinders'].unique())
print(data['Cylinders'].value_counts().sort_values())                              # This will give no imformation can be removed

In [ ]:
## Gears

print(data['Gears'].unique())
print(data['Gears'].value_counts().sort_values())                                        ## can remove car with gear 3,4

Numerical Data
1.Price,2.Age,3.KM,4.Weight

In [ ]:
plt.hist(data['Price'],bins=7)

In [ ]:
plt.hist(data['Age_08_04'],bins=7)

In [ ]:
plt.hist(data['KM'],bins=7)

In [ ]:
plt.hist(data['Weight'],bins=7)

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

In [ ]:
data=data.drop_duplicates()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
df=data.drop('Cylinders',axis=1)
df.head()

In [ ]:
new_df=df[df['cc'] != 16000]

In [ ]:
new_df[new_df['cc'] == 16000]

In [ ]:
new_df.shape

In [ ]:
sns.boxplot(new_df['Price'])


In [ ]:
sns.distplot(new_df['Price'])

skewness = new_df['Price'].skew()
print(f"Skewness of the feature: {skewness}")

In [ ]:
sns.boxplot(new_df['Age_08_04'])

In [ ]:
sns.distplot(new_df['Age_08_04'])

Age_skewness = new_df['Age_08_04'].skew()
print(f"Skewness of the feature: {Age_skewness}")

In [ ]:
sns.boxplot(new_df['KM'])

In [ ]:
sns.distplot(new_df['KM'])

KM_skewness = new_df['KM'].skew()
print(f"Skewness of the feature: {KM_skewness}")

In [ ]:
sns.boxplot(new_df['Weight'])

In [ ]:
sns.distplot(new_df['Weight'])

KM_skewness = new_df['Weight'].skew()
print(f"Skewness of the feature: {KM_skewness}")

In [ ]:
data_num=new_df.drop('Fuel_Type',axis=1)

In [ ]:
data_num

In [ ]:
data_num.corr() # # same scale corelation is better

In [ ]:
sns.heatmap(data_num.corr(),annot=True)

## Insights
# Price and Age  strong negative correlation
# Price and KM  negative correlation
# Price and Weight positive correlation

# cc and weight positive corelation

In [ ]:
q1=new_df['Price'].quantile(0.25)
print(q1)
q3=new_df['Price'].quantile(0.75)
print(q3)

In [ ]:
IQR=q3-q1
print(IQR)

In [ ]:
min_val=q1-1.5*IQR
max_val=q3+1.5*IQR

print(min_val,max_val)

In [ ]:
new_df_without_outliers=new_df[(new_df['Price'] >= min_val) & (new_df['Price'] <= max_val)]
new_df_without_outliers.shape

In [ ]:
outliers=new_df[(new_df['Price'] < min_val) | (new_df['Price'] > max_val)]
outliers.shape

In [ ]:
new_df.describe()

In [ ]:
new_df_without_outliers.describe()

In [ ]:
sns.distplot(new_df_without_outliers['Price'])

skewness = new_df_without_outliers['Price'].skew()
print(f"Skewness of the feature: {skewness}")

In [ ]:
plt.boxplot(new_df_without_outliers['Price'])

In [ ]:
print(new_df.head())
print(new_df.shape)

## **Model Building**

In [ ]:
new_df.shape

In [ ]:
new_df_without_outliers.shape

In [ ]:
target=new_df[['Price']]
target.shape

In [ ]:
target1=new_df_without_outliers[['Price']]
target1.shape

In [ ]:
features=new_df.drop('Price',axis=1)
features.shape

In [ ]:
features1=new_df_without_outliers.drop('Price',axis=1)
features1.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(features,target,train_size=0.80,random_state=100)

In [ ]:
x_train1,x_test1,y_train1,y_test1=train_test_split(features1,target1,train_size=0.80,random_state=100)                   ## without outliers

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
print(x_train1.shape)
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

**OneHotEncoder , StandardScaler**

In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [ ]:
ohe=OneHotEncoder(drop='first',dtype=np.int32)

In [ ]:
x_train_ohe=ohe.fit_transform(x_train[['Fuel_Type']]).toarray()
x_test_ohe=ohe.transform(x_test[['Fuel_Type']]).toarray()

In [ ]:
x_train_new=np.hstack((x_train[['Age_08_04','KM','HP','Automatic','cc','Doors','Gears','Weight']].values,x_train_ohe))
x_test_new=np.hstack((x_test[['Age_08_04','KM','HP','Automatic','cc','Doors','Gears','Weight']].values,x_test_ohe))

In [ ]:
x_test_new.shape

In [ ]:
std=StandardScaler()

In [ ]:
x_train_std=std.fit_transform(x_train_new)
x_test_std=std.transform(x_test_new)

In [ ]:
x_train_new.shape

# Model1

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.compose import make_column_selector

In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Step 1: Preprocess data (OneHotEncode categorical columns)
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(drop='first'), ['Fuel_Type'])  # OneHotEncoder for categorical columns
    ],
    remainder='passthrough'  # Keep numerical columns untouched for now
)

# Step 2: Define the pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),         # Apply preprocessing (OneHotEncoder)
    ('scaler', StandardScaler()),           # Scale all columns (both numerical and encoded)
    ('mlr', LinearRegression())             # Linear regression model
])

# Fit pipeline
pipe.fit(x_train, y_train)

# Evaluate the model
score = pipe.score(x_test, y_test)
print(f"Model R^2 score: {score:.2f}")

In [ ]:
pipe

In [ ]:
# Step 1: Preprocess data (OneHotEncode categorical columns)                                ## without outlier
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(drop='first'), ['Fuel_Type'])  # OneHotEncoder for categorical columns
    ],
    remainder='passthrough'  # Keep numerical columns untouched for now
)

# Step 2: Define the pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),         # Apply preprocessing (OneHotEncoder)
    ('scaler', StandardScaler()),           # Scale all columns (both numerical and encoded)
    ('mlr', LinearRegression())             # Linear regression model
])

# Fit pipeline
pipe.fit(x_train1, y_train1)

# Evaluate the model
score = pipe.score(x_test1, y_test1)
print(f"Model R^2 score: {score:.2f}")

# **Model2**

## PCA

In [ ]:
# Step 1: Preprocess data (OneHotEncode categorical columns)
preprocessor1 = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(drop='first'), ['Fuel_Type'])  # OneHotEncoder for categorical columns
    ],
    remainder='passthrough'  # Keep numerical columns untouched for now
)

# Step 2: Define the pipeline
pipe2 = Pipeline([
    ('preprocessor', preprocessor1),         # Apply preprocessing (OneHotEncoder)
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.97)),            # Scale all columns (both numerical and encoded)
    ('mlr', LinearRegression())             # Linear regression model
])

# Fit pipeline
pipe2.fit(x_train, y_train)

# Evaluate the model
score = pipe2.score(x_test, y_test)
print(f"Model R^2 score: {score:.2f}")

In [ ]:
pipe2

In [ ]:
# Step 1: Preprocess data (OneHotEncode categorical columns)
preprocessor1 = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(drop='first'), ['Fuel_Type'])  # OneHotEncoder for categorical columns
    ],
    remainder='passthrough'  # Keep numerical columns untouched for now
)

# Step 2: Define the pipeline
pipe2 = Pipeline([
    ('preprocessor', preprocessor1),         # Apply preprocessing (OneHotEncoder)
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.97)),            # Scale all columns (both numerical and encoded)
    ('mlr', LinearRegression())             # Linear regression model
])

# Fit pipeline
pipe2.fit(x_train1, y_train1)

# Evaluate the model
score = pipe2.score(x_test1, y_test1)
print(f"Model R^2 score: {score:.2f}")

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=0.95)

In [ ]:
pca.fit(x_train_new)
pca.fit(x_test_new)

In [ ]:
x_train_pca=pca.transform(x_train_new)
x_test_pca=pca.transform(x_test_new)

In [ ]:
print(x_train_pca.shape)
print(x_test_pca.shape)

In [ ]:
mlr=LinearRegression()

In [ ]:
mlr.fit(x_train_pca,y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_predict=mlr.predict(x_test_pca)
y_predict.shape

In [ ]:
y_predict.dtype

In [ ]:
y_predict = y_predict.astype('int64')

In [ ]:
y_predict.dtype

In [ ]:
y_test.values.dtype

In [ ]:
accuracy_score(y_test,y_predict)

## Model 3

# Regularisation

# Improvements

1. random state
2.

In [ ]:
ct2=ColumnTransformer(transformers=[('ohe',OneHotEncoder(categories=ohe.categories_,drop='first'),['Fuel_Type'])],remainder='passthrough')

In [ ]:
pipe2=make_pipeline(ct2,mlr)

In [ ]:
pipe2.fit(x_train,y_train)

In [ ]:
y_predict2=pipe.predict(x_test)

In [ ]:
r2_score(y_test,y_predict2)

# model2